<h1 align=center><font size = 5>Capstone Week 3 - Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Part 1 - Data Scraping of Postal Codes and Neighborhood Names

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# used for scraping
import requests
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## Part 1

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

To create the dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood<\li>
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
    

In [7]:
# Get the json from wikipedia and read it with beautiful soup
res = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

# The wikipedia page is not really a table, it's more like a matrix.  Concatenate all the text into a single column
df=df[0]
frames = [df.iloc[:,0],df.iloc[:,1],df.iloc[:,2],df.iloc[:,3],df.iloc[:,4],df.iloc[:,5],df.iloc[:,6],df.iloc[:,7]]
df = pd.concat(frames)
df = df.to_frame()
df.reset_index(inplace = True, drop=True) # Fix the indices
df.columns = ['raw']

# get rid of rows for postal codes that aren't assigned
df = df[~df.raw.str.contains("Not assigned")]

# Ditch the "enclaves" too.  They're related to bulk mailing, and not interesting for the purposes of this analysis, which is centered on fun places to live.  
# Also they don't follow exactly the same format and mess up the cleaning process.
df = df[~df.raw.str.contains("Enclave")]
df.reset_index(inplace = True, drop=True)

# The dataframe has only a single column that has all the data kludged together.  Regex would probably be prettier, but let's pull the data out without it...
# Postal code is the first three letters
df['PostalCode'] = df['raw'].str[:3] 
df['raw'] = df['raw'].str[3:-1] # -1 removes the closing parentheses that we don't need

# now split up the neighborhood and boroughs
df[['Neighborhood','Borough']]=df.raw.str.split("(",n=1,expand=True,)
del df["raw"]

df.head()

,PostalCode,Neighborhood,Borough
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df.shape

(90, 3)